In [4]:
from __future__ import print_function
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=4, suppress=True)
import cv2
import versor as vsr
from hand_eye_calibration import HandEyeCalibrator

Populating the interactive namespace from numpy and matplotlib


### Object points

In [9]:
pattern_size = (10,6)
square_size = 0.02
pattern_points = np.zeros( (np.prod(pattern_size), 3), np.float64)
pattern_points[:,:2] = np.indices(pattern_size).T.reshape(-1,2)
pattern_points *= square_size
print(pattern_points)

[[ 0.    0.    0.  ]
 [ 0.02  0.    0.  ]
 [ 0.04  0.    0.  ]
 [ 0.06  0.    0.  ]
 [ 0.08  0.    0.  ]
 [ 0.1   0.    0.  ]
 [ 0.12  0.    0.  ]
 [ 0.14  0.    0.  ]
 [ 0.16  0.    0.  ]
 [ 0.18  0.    0.  ]
 [ 0.    0.02  0.  ]
 [ 0.02  0.02  0.  ]
 [ 0.04  0.02  0.  ]
 [ 0.06  0.02  0.  ]
 [ 0.08  0.02  0.  ]
 [ 0.1   0.02  0.  ]
 [ 0.12  0.02  0.  ]
 [ 0.14  0.02  0.  ]
 [ 0.16  0.02  0.  ]
 [ 0.18  0.02  0.  ]
 [ 0.    0.04  0.  ]
 [ 0.02  0.04  0.  ]
 [ 0.04  0.04  0.  ]
 [ 0.06  0.04  0.  ]
 [ 0.08  0.04  0.  ]
 [ 0.1   0.04  0.  ]
 [ 0.12  0.04  0.  ]
 [ 0.14  0.04  0.  ]
 [ 0.16  0.04  0.  ]
 [ 0.18  0.04  0.  ]
 [ 0.    0.06  0.  ]
 [ 0.02  0.06  0.  ]
 [ 0.04  0.06  0.  ]
 [ 0.06  0.06  0.  ]
 [ 0.08  0.06  0.  ]
 [ 0.1   0.06  0.  ]
 [ 0.12  0.06  0.  ]
 [ 0.14  0.06  0.  ]
 [ 0.16  0.06  0.  ]
 [ 0.18  0.06  0.  ]
 [ 0.    0.08  0.  ]
 [ 0.02  0.08  0.  ]
 [ 0.04  0.08  0.  ]
 [ 0.06  0.08  0.  ]
 [ 0.08  0.08  0.  ]
 [ 0.1   0.08  0.  ]
 [ 0.12  0.08  0.  ]
 [ 0.14  0.08